In [1]:
%load_ext autoreload
%autoreload 2
import telepot
bot = telepot.Bot('436447719:AAGq_iK2hE3cPFhmL9Jh53g9HyVkw3SXZbg')
def qq(txt):
    bot.sendMessage(1114926, txt)

In [2]:
qq('start')

In [3]:
import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter
import pyarrow.parquet as pq
import pyarrow as pa

import pandas as pd
from tqdm.notebook import tqdm
import dill
import os
from pqdm.processes import pqdm
import re

from workers import *
    
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec

import random
import collections

In [18]:
df = pd.read_parquet('df_tarin.pq', columns=['vacancy_id', 'employer'])

In [6]:
# df.columns
# Index(['vacancy_id', 'name', 'description', 'area_id', 'creation_date',
#        'work_schedule', 'work_experience', 'compensation_from',
#        'compensation_to', 'currency', 'key_skills', 'employer',
#        'specializations', 'is_test', 'part', 'employment', 'spec_len',
#        'groups_of_spec', 'len_groups', 'html_tags', 'clear_text',
#        'processed_text'],
#       dtype='object')

In [17]:
df[df.employer == '0000026a6f11']

,vacancy_id,name,description,area_id,creation_date,work_schedule,work_experience,compensation_from,compensation_to,currency,...,specializations,is_test,part,employment,spec_len,groups_of_spec,len_groups,html_tags,clear_text,processed_text


In [21]:
df.shape

(1456325, 2)

In [36]:
empl = df['employer'].value_counts().to_dict()

In [33]:
df['is_validation'] = False

In [41]:
df['num_samples'] = df['employer'].apply(lambda x: empl[x])

In [43]:
validation = []
valid_part = 0.2
for k,i in empl.items():
    if i > 1:
        validation += df[df['employer'] == k]['vacancy_id'].sample(frac=valid_part, replace=False, random_state=42).to_list()

In [48]:
validation += df[df['num_samples'] == 1]['vacancy_id'].sample(frac=valid_part, replace=False, random_state=42).to_list()

In [51]:
dill.dump(validation, open('validatin.bin', 'wb'))